In [51]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../../")

from dataloader import SimpleDataloader
from params import ATTN_SEQ2SEQ_PARAMS
from models.attn_seq2seq import Seq2seq, count_parameters, Encoder, Decoder
from utils import (
    save_model,
    get_torch_device,
    epoch_time,
    arg_copy,
    save_to_artifact,
    save_test_df,
    save_metrics,
)
from test_metrics.test_model import Model_tester

import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import numpy as np
import math
import sys
import time
import glob
import argparse
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
hp = ATTN_SEQ2SEQ_PARAMS


In [14]:
device = get_torch_device()


data = SimpleDataloader(**vars(hp))
train_dataloader = data.get_train_dataloader()
val_dataloader = data.get_val_dataloader()
test_dataloader = data.get_test_dataloader()


Loading Form Cache


In [52]:

model = Seq2seq(**vars(hp))
model.init_weights()
print(f"The model has {count_parameters(model):,} trainable parameters")
model.to(device)

The model has 112,763,401 trainable parameters


Seq2seq(
  (encoder): Encoder(
    (embedding): Embedding(45000, 300)
    (rnn): LSTM(300, 600, num_layers=2, dropout=0.3, bidirectional=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(45000, 300)
    (rnn): LSTM(300, 1200, num_layers=2, dropout=0.3)
    (fc): Linear(in_features=1200, out_features=45000, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [53]:
it=iter(val_dataloader)
src,trg,src_len=next(it)
src = src.to(device) 
trg = trg.to(device)

In [54]:
pred = model(src, trg)

torch.Size([40, 64])
torch.Size([40, 64, 300])
torch.Size([40, 64, 1200]) torch.Size([4, 64, 600]) torch.Size([4, 64, 600])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torch.Size([2, 64, 1200]) torch.Size([2, 64, 1200])
Decoder
torch.Size([64]) torc

In [56]:
pred.shape

torch.Size([40, 64, 45000])

In [33]:
a=torch.zeros(4,64,600)

In [34]:
a.shape

torch.Size([4, 64, 600])

In [42]:
torch.cat((a[0:2,:,:],a[2:,:,:]),dim=2).shape

torch.Size([2, 64, 1200])